In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2,preprocess_input
from tensorflow.keras.layers import Dense,Conv2D,GlobalAvgPool2D,Input
from tensorflow.keras.preprocessing.image import load_img,ImageDataGenerator
from tensorflow.keras import callbacks,optimizers
import numpy as np
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/MyDrive/Dog_classification/

[Errno 2] No such file or directory: 'drive/MyDrive/Dog_classification/'
/content/drive/MyDrive/Dog_pro


In [ ]:
!unzip Dog_Dataset.zip

In [ ]:
os.listdir("Dog_Dataset")

In [ ]:
!ls Dog_Dataset/

In [ ]:
for i in os.listdir("Dog_Dataset"):
  print(i,len(os.listdir("Dog_Dataset/"+i)))

In [ ]:
try:
  os.mkdir("train")
  os.mkdir("test")
except:
  pass
for i in os.listdir("Dog_Dataset"):
  try:
    os.mkdir("train/"+i)
    os.mkdir("test/"+i)
  except:
    pass
  for j in os.listdir("Dog_Dataset/"+i)[:1000]:
    os.rename("Dog_Dataset/"+i+"/"+j,"train/"+i+"/"+j)
  for j in os.listdir("Dog_Dataset/"+i)[:400]:
    os.rename("Dog_Dataset/"+i+"/"+j,"test/"+i+"/"+j)

In [ ]:
for i in os.listdir("test"):
  print(i,len(os.listdir("test/"+i)))

001.Affenpinscher 19
002.Afghan_hound 15
003.Airedale_terrier 15
004.Akita 18
005.Alaskan_malamute 32
006.American_eskimo_dog 19
007.American_foxhound 15
008.American_staffordshire_terrier 21
009.American_water_spaniel 0
010.Anatolian_shepherd_dog 15
011.Australian_cattle_dog 21
012.Australian_shepherd 21
013.Australian_terrier 11
014.Basenji 24
015.Basset_hound 28
016.Beagle 15
017.Bearded_collie 17
018.Beauceron 15
019.Bedlington_terrier 13
020.Belgian_malinois 17
021.Belgian_sheepdog 19
022.Belgian_tervuren 12
023.Bernese_mountain_dog 20
024.Bichon_frise 17
025.Black_and_tan_coonhound 2
026.Black_russian_terrier 6
027.Bloodhound 19
028.Bluetick_coonhound 0
029.Border_collie 29
030.Border_terrier 15
031.Borzoi 15
032.Boston_terrier 20
033.Bouvier_des_flandres 10
034.Boxer 19
035.Boykin_spaniel 15
036.Briard 20
037.Brittany 15
038.Brussels_griffon 15
039.Bull_terrier 24
040.Bulldog 15
041.Bullmastiff 24
042.Cairn_terrier 18
043.Canaan_dog 15
044.Cane_corso 19
045.Cardigan_welsh_corgi 

In [ ]:
def img_Data(dir_path,target_size,batch,class_lst,preprocessing,):
  if preprocessing:
    gen_object =ImageDataGenerator(preprocessing_function=preprocessing)
  else:
    gen_object =ImageDataGenerator()

  return(gen_object.flow_from_directory(dir_path,
                                          target_size= target_size,
                                          batch_size=batch,
                                          class_mode='sparse',
                                          classes=class_lst,
                                          shuffle=True))

In [ ]:
train_data_gen =img_Data("train",(224,224),500,os.listdir("train"),preprocess_input)
valid_data_gen =img_Data("test",(224,224),500,os.listdir("test"),preprocess_input)

Found 6680 images belonging to 133 classes.
Found 1633 images belonging to 133 classes.


In [ ]:
base_model=tf.keras.applications.mobilenet_v2.MobileNetV2(
              input_shape=(224,224,3),
              alpha=1.0,
              include_top=False,
              weights='imagenet',
              input_tensor=None,
              pooling=None,
              classes=1000,
              classifier_activation='softmax')

9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
base_model.trainable=False

In [ ]:
model=tf.keras.models.Sequential()
model.add(base_model)
model.add(GlobalAvgPool2D())
model.add(Dense(1024,activation='relu'))
model.add(Dense(1024,activation='softmax'))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
elst=callbacks.EarlyStopping(monitor='val_loss',patience=5,mode='min')
save_ck=callbacks.ModelCheckpoint('.mdl_wt.hdf5',save_best_only=True,monitor='val_loss',mode='min')

In [ ]:
model.fit(train_data_gen,batch_size=500,validation_data=valid_data_gen,callbacks=[elst,save_ck],epochs=10)

In [ ]:
import os
from os import listdir
from PIL import Image

categ = ['Dog_Dataset', 'train', 'test']
dataset = '/content/drive/MyDrive/Dog_pro'

for cat in categ:
  img_path = os.path.join(dataset, cat)
  for foldername in listdir(img_path):
    sign_path = os.path.join(img_path, foldername)
    print(sign_path)
    for sign in listdir(sign_path):
      if sign.endswith('.jpg'):
        try:
          img = Image.open(os.path.join(sign_path, sign)) # open the image file
          img.verify() # verify that it is, in fact an image
        except (IOError, SyntaxError) as e:
          print('Bad file:', sign)

In [ ]:
model.fit(train_data_gen,batch_size=500,validation_data=valid_data_gen,callbacks=[elst,save_ck],epochs=20)

Epoch 1/20
 5/14 [=========>....................] - ETA: 3:07 - loss: 0.7213 - accuracy: 0.7944

UnknownError: ignored

In [ ]:
model.fit(train_data_gen, epochs=10, validatio n_data=valid_data_gen,validation_steps=1)